# Astellas - Accumulations
## Next year
* Manually

In [2]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [5]:
df_export = pd.DataFrame(columns=fix_columns_accumulations[:-1])

# Add HCP
df_export = add_accumulation(df_export, AccType.hcp_amount,
                             registration_fees = '680,40 CHF',
                             travel_accommodation = '794,30 CHF',
                             fees = '6 436,60 CHF',
                             related_expenses = '8 354,91 CHF',
                             total = '16 266,21 CHF'
                            )

# Add HCO
df_export = add_accumulation(df_export, AccType.hco_amount,
                            donations_grants = '10 000,00 CHF',
                            sponsorship = '23 870,00 CHF',
                            total = '33 870,00 CHF')

# RnD
df_export = add_accumulation(df_export, AccType.rnd,
                            total = '667 865,59 CHF')

#Numberize and sum
df_export = replace_comma_to_dot(df_export)
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)
#df_export = sum_amounts(df_export)

#Fix RnD
df_export.loc[df_export.type == 'rnd', 'donations_grants'] = np.NaN

export_acumulations(df_export, 'astellas')

hcp_count not found or not unique
hco_count not found or not unique
saved
